In [1]:
import os
import pandas as pd
import cv2

deleted_range = []
right_side = []
vdo_file = None
right_ep_filename = None

def render_ui_frame_id(frame, frame_id, frame_total):
    cv2.rectangle(frame, (10, 5), (400, 35), (0,0,0), -1)
    cv2.putText(frame, f'{frame_total} : {frame_id}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

def render_index(frame,current_idx,total_idx):
    cv2.putText(frame, f'{current_idx}/{total_idx}', (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

def show_vdo(_ep_id):
    session_id = vdo_file[:16] + r'_scaled'
    # __episode_dir = _ep_id
    # os.path.join(__episode_dir, f"{_ep_id}{'.csv'}")
    # episode_path = os.path.join(__episode_dir, f"{_ep_id}{'.csv'}")
    episode_path = _ep_id
    print(episode_path)
    df = pd.read_csv(episode_path, usecols=['start', 'end'])
    df_list = df.values.tolist()
    video_path = os.path.join(os.path.join(os.getcwd(), 'Dataset'), f"{session_id}.MOV")
    # print(video_path)
    capture_handler = cv2.VideoCapture(video_path)
    if not capture_handler.isOpened():
        print(f"Error: Unable to open the video file {video_path}")
        exit()
    # Get the total frame of the video%
    frame_total = int(capture_handler.get(cv2.CAP_PROP_FRAME_COUNT))
    # Define the window name, also use it as tag for using in CV
    window_name = f"{session_id} - mICE Video Marker"
    # Create a window
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    frame_id_current = 0
    saved_frame = None
    flag_pause = True
    is_on_seeking = False
    cnt_episode_frame = 0
    flag = True
    total_index = len(df_list)
    while flag ==True:
        try:
        # Check if the video is paused, if so, then keep playing the current frame
            if is_on_seeking:
                 if flag_is_set_marker_founded:
                    is_on_seeking = False
                    flag_is_set_marker_founded = False
                    flag_pause = True
                    saved_frame = None
                    capture_handler.set(cv2.CAP_PROP_POS_FRAMES, frame_id_current)
            if flag_pause:
                if saved_frame is None:
                    ret, saved_frame = capture_handler.read()
                frame = saved_frame.copy()
            else:
                ret, frame = capture_handler.read()
            frame_id_current = int(capture_handler.get(cv2.CAP_PROP_POS_FRAMES)) - 1
            # Render frame info
            render_ui_frame_id(frame, frame_id_current, frame_total)
            render_index(frame,cnt_episode_frame,total_index)
            # Display the frame
            cv2.imshow(window_name, frame)
            # Wait for a key press, q to exit, space to start/stop the video (keep playing current frame)
            key = cv2.waitKey(1) & 0xFF
            if key in [ord('q'),ord('Q')]:
                cv2.destroyAllWindows()
                flag = False
            elif key in [ord('R'),ord('r'),ord('d'),ord('D'),ord(' '),ord('Z'),ord('z'),ord('x'),ord('v'),ord('b'),ord('n'),ord ('m'),ord('e'),ord('E'),ord('r'),ord('R'),ord('t'),ord('T')]:
                if key == ord(' '):
                    is_on_seeking = False
                    if flag_pause:
                        flag_pause = False
                        saved_frame = None
                    else:
                        flag_pause = True
                    frame_id_current = int(capture_handler.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                if key==ord('r') or key==ord('R'):
                    cnt_episode_frame -=1
                    frame_id_current = df_list[cnt_episode_frame][0] 
                if key==ord('d') or key==ord('D'):
                    cnt_episode_frame +=1
                    frame_id_current = df_list[cnt_episode_frame][0] 
                if key==ord('e') or key==ord('E'):
                    if(frame_id_current!=df_list[cnt_episode_frame][0]):
                        frame_id_current = df_list[cnt_episode_frame][0]
                    deleted_range.append(frame_id_current)
                    print(deleted_range[-1])
                if key==ord('w') or key==ord('W'):
                    del deleted_range[-1]
                    for i in range(0, len(deleted_range), 10):
                        print(*deleted_range[i:i+10])
                if key==ord('t') or key==ord('T'):
                    if(frame_id_current!=df_list[cnt_episode_frame][0]):
                        frame_id_current = df_list[cnt_episode_frame][0]
                    right_side.append(frame_id_current)
                    print(right_side)
                if key == ord('Z'):
                    frame_id_current = 0
                if key == ord('z'):
                    frame_id_current -= 240
                if key == ord('x'):
                    frame_id_current -= 1
                if key == ord('v'):
                    frame_id_current += 1
                if key == ord('b'):
                    frame_id_current += 240
                if key == ord('n'):
                    frame_id_current += 120
                if key == ord('m'):
                    frame_id_current += 500
                capture_handler.set(cv2.CAP_PROP_POS_FRAMES, frame_id_current)
                saved_frame = None
        except Exception as e:
            print(f"Error in {e}")
            continue

In [7]:
right_dir = './output_episodes'
output_dir = './right_clean'
selected_file = '0eb579b7e128131'
for i, file in enumerate(os.listdir(right_dir)[:]):
    if file[0:15] == selected_file[0:15]:
        selected_file = file
        right_ep_filename = os.path.join(right_dir, file)
        right_clean_filename = os.path.join(output_dir,r'right_'+file)
        vdo_file = file
        right_ep_df = pd.read_csv(right_ep_filename)
        print(right_ep_filename)
        print(f'len right:{len(right_ep_df)}')


./output_episodes\0eb579b7e1281316_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv
len right:283


In [8]:
show_vdo(right_ep_filename)
right_df = right_ep_df[~right_ep_df['start'].isin(deleted_range)]
print(len(right_df))
print(deleted_range)
right_df.to_csv(right_clean_filename,index=False)

./output_episodes\0eb579b7e1281316_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv
236134
13308
23224
32617
33201
33971
34397
73321
86419
101037
106685
106935
107484
107863
110221
111584
122121
122204
123026
151144
151494
151601
190899
203996
204079
204149
204618
204727
211474
233705
236134
240641
Error in list index out of range
Error in list index out of range
252
[295, 9149, 23683, 24578, 44573, 47158, 47294, 51317, 59332, 96925, 115245, 143191, 145010, 146817, 184819, 185168, 193532, 197835, 209389, 215255, 236134, 13308, 23224, 32617, 33201, 33971, 34397, 73321, 86419, 101037, 106685, 106935, 107484, 107863, 110221, 111584, 122121, 122204, 123026, 151144, 151494, 151601, 190899, 203996, 204079, 204149, 204618, 204727, 211474, 233705, 236134, 240641]


In [9]:
show_vdo(right_clean_filename)

./right_clean\right_0eb579b7e1281316_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv
103369
151549
Error in list index out of range
